In [1]:

import os
import pandas as pd
import time
import numpy as np
import re


In [2]:
# Paths
DATA_PATH ="../../data"

path_lb_embb = os.path.join(DATA_PATH, "lb_npy.npy")
path_dataset = os.path.join(DATA_PATH, "spotify_dataset_sin_duplicados_4.csv")


## Revisando que sean embbedings iguales en khipu

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

MODEL_NAME = 'brunokreiner/lyrics-bert'
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)


/home/marcelino.maita/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Sentence embeddings:
tensor([[ 8.5982e-02, -1.9938e-01,  3.1403e-01, -2.1909e-01, -3.9347e-04,
          2.8418e-01,  1.1639e+00, -2.5347e-01, -5.9031e-01,  2.8978e-01,
         -4.3361e-01,  2.6265e-01,  1.2186e-02, -2.1822e-01,  1.0540e-01,
          1.8403e-01,  5.8598e-01, -1.6739e-01, -2.6611e-01,  3.8995e-01,
         -1.9267e-01,  3.5165e-01,  5.3251e-01,  4.8379e-01,  5.9596e-01,
         -1.0685e+00,  3.0516e-01, -4.1388e-01,  3.8347e-01, -3.3050e-01,
         -3.3174e-01,  4.0777e-01,  1.8344e-01, -2.9582e-01, -1.4029e-01,
         -4.2408e-01,  2.6159e-01, -2.7347e-01, -7.7970e-02, -3.2933e-01,
          3.9029e-01,  8.3547e-02,  1.7815e-01, -4.2287e-01,  2.5291e-02,
          5.4537e-01, -3.9749e-01,  1.0715e-01, -1.1429e+00,  4.7099e-01,
          2.0548e-01,  1.8556e-01, -3.1428e-01,  6.5785e-01,  2.4795e-01,
          5.3135e-01, -1.9034e-01,  3.3224e-01,  4.9253e-01, -1.7496e-02,
          4.9282e-01,  5.9564e-01,  2.2089e-01,  1.9639e-01, -1.7313e-01,
         -2.4567e

/home/marcelino.maita/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [4]:
BATCH_SIZE = 500 # Porque nos quedamos sin RAM :(
INIT = 0
TOTAL_ROWS = 108138
# TOTAL_ROWS = 1000

# To save vectors
# save_dir = "/content/drive/MyDrive/embeddings_lyricsbert_to_fusion/"
save_dir = os.path.join(DATA_PATH, "embbedings_khipu")
os.makedirs(save_dir, exist_ok=True)
# start = 0
# end = 5000
# ROWS = end - start
A = ['text', 'song', 'Artist(s)', 'Album', 'Similar Artist 1', 'Genre']


for start in range(INIT, TOTAL_ROWS, BATCH_SIZE):
  end = min(start + BATCH_SIZE, TOTAL_ROWS)
  print(f"\nProcesando filas {start} a {end-1}...")
  df = pd.read_csv(path_dataset, skiprows=range(1, start + 1), nrows=end - start)
  
  df['combined_text'] = df[A].fillna('').agg(' '.join, axis=1)

  df_sentences = df['combined_text'].fillna("").astype(str).tolist()

  print("shape de sentences: ", len(df_sentences))

  # Tokenize input
  start_time = time.time()

  encoded_df_input = tokenizer(df_sentences, padding=True, truncation=True, return_tensors='pt')

  # Compute embeddings
  with torch.no_grad():
      model_output = model(**encoded_df_input)

  # Pooling (mean)
  sentence_df_embeddings = mean_pooling(model_output, encoded_df_input['attention_mask'])



  # Print results
  end_time = time.time()
  print(f"\nTiempo total: {end_time - start_time:.2f} segundos")

  print("Iniciando guardado de los vectores: ")
  embeddings_np = sentence_df_embeddings.numpy()

  # Guardar en formato binario .npy
  npy_filename = f"embeddings_lyricsbert_{start}_{end-1}.npy"
  np.save(os.path.join(save_dir, npy_filename), embeddings_np)

  # # Guardar en CSV
  # csv_filename = f"embeddings_lyricsbert_{start}_{end-1}.csv"
  # np.savetxt(os.path.join(save_dir, csv_filename), embeddings_np, delimiter=",")
  print("Sentence embeddings:")
  # print(sentence_df_embeddings)
  print(sentence_df_embeddings.shape)
  print(f"Guardado batch {start}-{end-1}")



Procesando filas 0 a 499...
shape de sentences:  500


/home/marcelino.maita/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Tiempo total: 5.92 segundos
Iniciando guardado de los vectores: 
Sentence embeddings:
torch.Size([500, 300])
Guardado batch 0-499

Procesando filas 500 a 999...
shape de sentences:  500


/home/marcelino.maita/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Tiempo total: 4.54 segundos
Iniciando guardado de los vectores: 
Sentence embeddings:
torch.Size([500, 300])
Guardado batch 500-999

Procesando filas 1000 a 1499...
shape de sentences:  500


...


Tiempo total: 4.53 segundos
Iniciando guardado de los vectores: 
Sentence embeddings:
torch.Size([500, 300])
Guardado batch 107000-107499

Procesando filas 107500 a 107999...
shape de sentences:  500


/home/marcelino.maita/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Tiempo total: 4.48 segundos
Iniciando guardado de los vectores: 
Sentence embeddings:
torch.Size([500, 300])
Guardado batch 107500-107999

Procesando filas 108000 a 108137...
shape de sentences:  138


/home/marcelino.maita/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Tiempo total: 1.29 segundos
Iniciando guardado de los vectores: 
Sentence embeddings:
torch.Size([138, 300])
Guardado batch 108000-108137


In [5]:
## Make fusion of the embeddings of example
save_dir_df_npy = os.path.join(save_dir, "lb_khipu_A.npy")
# print(os.listdir(save_dir))

embbedings_df_npy =os.listdir(save_dir)

def get_start_number(filename):
    match = re.search(r'embeddings_lyricsbert_(\d+)_\d+\.npy', filename)
    return int(match.group(1)) if match else float('inf')
embbedings_df_npy = sorted(embbedings_df_npy, key=get_start_number)

all_embeddings = []

for embb in embbedings_df_npy:
  if embb.endswith('.npy'):
    # print(embb)
    file_path = os.path.join(save_dir, embb)
    embeddings = np.load(file_path)
    all_embeddings.append(embeddings)
    print(f"Leído: {embb} - Shape: {embeddings.shape}")
    print(f"Total: {len(all_embeddings)}")

final_embeddings = np.vstack(all_embeddings)
print("Shape final:", final_embeddings.shape)

# Guardar en .npy
np.save(save_dir_df_npy, final_embeddings)
print(f"Embeddings guardados en: {save_dir_df_npy}")



Leído: embeddings_lyricsbert_0_499.npy - Shape: (500, 300)
Total: 1
Leído: embeddings_lyricsbert_500_999.npy - Shape: (500, 300)
Total: 2
Leído: embeddings_lyricsbert_1000_1499.npy - Shape: (500, 300)
Total: 3
Leído: embeddings_lyricsbert_1500_1999.npy - Shape: (500, 300)
Total: 4
Leído: embeddings_lyricsbert_2000_2499.npy - Shape: (500, 300)
Total: 5
Leído: embeddings_lyricsbert_2500_2999.npy - Shape: (500, 300)
Total: 6
Leído: embeddings_lyricsbert_3000_3499.npy - Shape: (500, 300)
Total: 7
Leído: embeddings_lyricsbert_3500_3999.npy - Shape: (500, 300)
Total: 8
Leído: embeddings_lyricsbert_4000_4499.npy - Shape: (500, 300)
Total: 9
Leído: embeddings_lyricsbert_4500_4999.npy - Shape: (500, 300)
Total: 10
Leído: embeddings_lyricsbert_5000_5499.npy - Shape: (500, 300)
Total: 11
Leído: embeddings_lyricsbert_5500_5999.npy - Shape: (500, 300)
Total: 12
Leído: embeddings_lyricsbert_6000_6499.npy - Shape: (500, 300)
Total: 13
Leído: embeddings_lyricsbert_6500_6999.npy - Shape: (500, 300)
Tot

In [6]:
#Delete the butches files

dir_clean = save_dir
print(dir_clean)

butches_embb = os.listdir(dir_clean)

for butch in butches_embb:
    if butch != "lb_khipu_A.npy" and butch != "lb_khipu.npy"  :
        # Delete 
        file_path = os.path.join(dir_clean, butch)
        print(file_path)
        os.remove(file_path)


../../data/embbedings_khipu
../../data/embbedings_khipu/embeddings_lyricsbert_1000_1499.npy
../../data/embbedings_khipu/embeddings_lyricsbert_500_999.npy
../../data/embbedings_khipu/embeddings_lyricsbert_36500_36999.npy
../../data/embbedings_khipu/embeddings_lyricsbert_45500_45999.npy
../../data/embbedings_khipu/embeddings_lyricsbert_63000_63499.npy
../../data/embbedings_khipu/embeddings_lyricsbert_26500_26999.npy
../../data/embbedings_khipu/embeddings_lyricsbert_42000_42499.npy
../../data/embbedings_khipu/embeddings_lyricsbert_63500_63999.npy
../../data/embbedings_khipu/embeddings_lyricsbert_58000_58499.npy
../../data/embbedings_khipu/embeddings_lyricsbert_86500_86999.npy
../../data/embbedings_khipu/embeddings_lyricsbert_68000_68499.npy
../../data/embbedings_khipu/embeddings_lyricsbert_60500_60999.npy
../../data/embbedings_khipu/embeddings_lyricsbert_69000_69499.npy
../../data/embbedings_khipu/embeddings_lyricsbert_83000_83499.npy
../../data/embbedings_khipu/embeddings_lyricsbert_1000

## 1) LB KHIPU VS LB COLAB

In [ ]:
# colab_embbeding = np.load(path_lb_embb) 
# khipu_embbeding = np.load("../data/embbedings_khipu/lb_khipu.npy") 
# are_equal = np.array_equal(colab_embbeding, khipu_embbeding)
# print("¿Son exactamente iguales?", are_equal)

# are_close = np.allclose(colab_embbeding, khipu_embbeding, atol=1e-6)
# print("¿Son prácticamente iguales (con tolerancia)?", are_close)


¿Son exactamente iguales? False
¿Son prácticamente iguales (con tolerancia)? True


Son iguales pero diferentes, con tol de e-6

## 2) Comparing df 1000 VS 1000 primeras filas de Colab an khipu

In [ ]:
# df_data = pd.read_csv(path_dataset)

In [ ]:
# df_data = df_data[: 1000]
# print(df_data.shape)
# list_text  = df_data['text'].fillna("").astype(str).tolist()
# print(len(list_text))


(1000, 41)
1000


In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torch


# #Mean Pooling - Take attention mask into account for correct averaging
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# # Sentences we want sentence embeddings for
# sentences = list_text

# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModel.from_pretrained(MODEL_NAME)

# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling. In this case, mean pooling.
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# print("Sentence embeddings:")
# print(sentence_embeddings)


/home/marcelino.maita/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Sentence embeddings:
tensor([[ 0.0043, -0.2640, -0.0681,  ..., -0.1682, -0.0212, -0.0591],
        [ 0.0013, -0.1634, -0.1466,  ..., -0.1082, -0.0870,  0.0344],
        [ 0.0144,  0.0016,  0.0267,  ..., -0.1320, -0.0767, -0.0183],
        ...,
        [ 0.1645, -0.0803, -0.1348,  ..., -0.2597, -0.2391,  0.0845],
        [ 0.1008, -0.1003,  0.0754,  ..., -0.1176, -0.1607,  0.1153],
        [ 0.0109, -0.2829,  0.1105,  ..., -0.0596, -0.1596,  0.0998]])


In [ ]:
# colab_embbeding_first = colab_embbeding[:1000]
# khipu_embbeding_first = khipu_embbeding[:1000]
# are_close_khipu = np.allclose(khipu_embbeding_first, sentence_embeddings, atol=1e-7)
# print("Son casi iguales con Khipu" if are_close_khipu else "Son diferentes")

# are_close_colab = np.allclose(colab_embbeding_first, sentence_embeddings, atol=1e-7)
# print("Son casi iguales con Colab" if are_close_colab else "Son diferentes")



Son casi iguales con Khipu
Son casi iguales con Colab


Ahora con los 1000 ultimos

In [ ]:
# # Actualziar df_data skdjksl

# df_data = pd.read_csv(path_dataset)
# df_data =df_data[-1000:] 
# print(df_data.shape)
# list_text_last  = df_data['text'].fillna("").astype(str).tolist()
# print(len(list_text_last))


(1000, 41)
1000


In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torch


# #Mean Pooling - Take attention mask into account for correct averaging
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# # Sentences we want sentence embeddings for
# sentences = list_text_last

# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModel.from_pretrained(MODEL_NAME)

# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling. In this case, mean pooling.
# sentence_embeddings_last = mean_pooling(model_output, encoded_input['attention_mask'])

# print("Sentence embeddings:")
# # print(sentence_embeddings)

# #               total        used        free      shared  buff/cache   available
# # Mem:          1.0Ti        23Gi       820Gi       3.2Gi       163Gi       975Gi
# # Swap:            0B          0B          0B
# # [marcelino.maita@ds001 sanity_check]$


/home/marcelino.maita/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Sentence embeddings:


In [ ]:
# print(colab_embbeding.shape)
# print(khipu_embbeding.shape)

(108138, 300)
(108138, 300)


In [ ]:
# colab_embbeding_last = colab_embbeding[-1000:]
# khipu_embbeding_last = khipu_embbeding[-1000:]
# are_close_khipu = np.allclose(khipu_embbeding_last, sentence_embeddings_last, atol=1e-7)
# print("Son casi iguales con Khipu" if are_close_khipu else "Son diferentes")

# are_close_colab = np.allclose(colab_embbeding_last, sentence_embeddings_last, atol=1e-7)
# print("Son casi iguales con Colab" if are_close_colab else "Son diferentes")



Son casi iguales con Khipu
Son casi iguales con Colab


En resumen son iguales con almenos tolerancia de 1e-6